In [1]:
import torch
import numpy as np
import pickle

def post_process(out):
    thd = 0.7
    thd2 = 0.6
    thrld = np.ones((1,6))*thd
    thrld_ten = torch.from_numpy(thrld).float()
    
    out = torch.FloatTensor(out)
    pred1 = (out > thrld_ten.expand(out.size())).float()
    max_idx = torch.argmax(out, 1, keepdim=True)
    one_hot = torch.zeros(out.size())
    pred2 = one_hot.scatter_(1, max_idx, 1)
    pred = (pred1+pred2 >= 1).float()

    pred = pred.tolist()
    out = out.tolist()
    for i in range(len(pred)):
        tmp = sum(pred[i])
        if tmp == 0.0:
            assert 1 == 0, print("NO PREDICTION ERROR")
        elif tmp == 1.0:
            idx = pred[i].index(1.0)
            if idx == 0:
                if out[i][1] > thd2:
                    pred[i][1] = 1.0
                    thd3 = out[i][1]
                    for j in range(5):
                        if out[i][j] > thd3:
                            pred[i][j] = 1.0
            elif 1 <= idx <= 3:
                choose = idx-1 if out[i][idx-1] >= out[i][idx+1] else idx+1
                not_choose = idx+1 if choose == idx-1 else idx-1
                if out[i][choose] > thd2:
                    pred[i][choose] = 1.0
                    thd3 = out[i][choose]
                    for j in range(5):
                        if out[i][j] > thd3:
                            pred[i][j] = 1.0
                if out[i][not_choose] > thd2:
                    pred[i][not_choose] = 1.0
            elif idx == 4:
                if out[i][3] > thd2:
                    pred[i][3] = 1.0
                    thd3 = out[i][3]
                    for j in range(5):
                        if out[i][j] > thd3:
                            pred[i][j] = 1.0
        elif tmp > 1.5 and pred[i][5] == 1.0:
            pred[i][5] = 0.0
            
        thd4 = 0.62
        if sum(pred[i]) == 2.0:
            if sum(pred[i]*np.array([32, 16, 8, 4, 2, 1])) == 6.0:
                if out[i][3] < thd4 or out[i][4] < thd4:
                    pred[i][3] = 0.0
                    pred[i][4] = 0.0
                    if out[i][3] > out[i][4]:
                        pred[i][3] = 1.0
                    else:
                        pred[i][4] = 1.0
                        
        thd5 = 0.59
        if sum(pred[i]) == 1.0:
            if pred[i][0] == 1.0 and out[i][1] > thd5:
                pred[i][1] = 1.0
            elif pred[i][1] == 1.0 and out[i][0] > thd5:
                pred[i][0] = 1.0
                    
    return pred

def vote(results, method="vote"):
    if method == "vote":
        print("postprocessing...")
        preds = [result[:131166]+post_process(result[131166:]) for result in tqdm(results)]
        print("voting!")
        thd = len(preds)//2
        ret = torch.FloatTensor(preds)
        print(ret.size())
        ret = (torch.sum(ret, 0) > thd).long().tolist()
        return ret
    elif method == "average":
        results = torch.FloatTensor(results)
        means = torch.mean(results, 0).tolist()
        ret = torch.LongTensor(post_process(means[:131166])).tolist() + \
              torch.LongTensor(means[131166:]).tolist()
        return ret
    else:
        assert 1 == 0, print("METHOD ERROR")

In [2]:
import pickle
import csv
import sys
import os
import random
import numpy as np
import multiprocessing as mp
from tqdm.notebook import tqdm

results = []
files = []
keeps = []
firsts = []
seconds = []
col_names = []
path = "./results/task1/"
lower = 0.735
upper = 0.75
method = "vote"

with open("seed2f1_task1.pkl", "rb") as f:
    seed2f1 = pickle.load(f)

for seed in list(seed2f1.keys()):
    if lower <= seed2f1[seed] <= upper:
        keeps.append(seed)

print("total number of csv file:", len(os.listdir(path))-1)
        
for file in os.listdir(path):
    if file[-4:] == ".csv":
        try:
            if int(file[7:-4])%100 == 87:
                firsts.append(file)
            elif int(file[7:-4])%100 == 13:
                seconds.append(file)
        except:
            pass
            
goods = ['result_22713.csv', 'result_813.csv', 'result_2913.csv', 'result_20813.csv',
         'result_22813.csv', 'result_23813.csv', 'result_8013.csv', 'result_2613.csv',
         'result_12113.csv', 'result_3813.csv', 'result_11013.csv', 'result_4013.csv']
files = firsts + goods
            
# assert len(keeps) == len(files)
print("remain number of csv file:", len(files))
ids = []

print("getting ids...")
with open(path+files[0], newline="") as f:
    rows = csv.reader(f)
    for i, row in enumerate(rows):
        if i == 0:
            col_names = row
        else:
            ids.append(row[0])
    
print("getting results...")
def get_results(file):
    result = []
    with open(path+file, newline="") as f:
        rows = csv.reader(f)
        for i, row in enumerate(rows):
            if i != 0:
                tmp = [float(x) for x in row[1:]]
                result.append(tmp)
    return result

with mp.Pool(processes=mp.cpu_count()) as pool:
    results = pool.map(get_results, tqdm(files))
    
votes = vote(results, method)

print("writing to file...")
with open("result_{}_{}.csv".format(method, len(files)), "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(col_names)
    for i in tqdm(range(len(results[0]))):
        w.writerow([ids[i]]+votes[i])

total number of csv file: 95
remain number of csv file: 95
getting ids...
getting results...



postprocessing...



voting!
torch.Size([95, 262948, 6])
writing to file...
